In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import librosa
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from tensorflow.keras.models import load_model

# ====== 고정 설정 ======
MODEL_PATH = "animal.keras"   # 저장해둔 모델 파일
THRESHOLD = 0.27
MAX_TOKEN_LENGTH = 256
SAMPLE_RATE = 16000
N_MELS = 128

# ====== 라벨(학습과 동일) ======
group_2point = [
    "고양이","사자","호랑이","코뿔소","고래","원숭이","닭","스컹크","미어캣","코알라",
    "앵무새","독수리","악어","늑대","까마귀","까치","비둘기","황새","외기러기","참새",
    "코끼리","양","잠자리","나비","공룡","곰","상어","얼룩말","하마","개","염소","산양",
    "나무늘보","족제비","여우","산토끼","기린","너구리","오소리","사슴","고라니","두루미",
    "멧돼지","캥거루","퓨마","표범","하이에나","치타","판다","오랑우탄","낙타","고릴라",
    "공작새","꿩","뱀","거북이","박쥐","노루","개구리","당나귀","젖소","한우","개미","제비",
    "물소","침팬지","펭귄","송아지","쥐","다람쥐","말","소"
]
group_1point = ["우랑우탄","포랑이","오랑이","도끼리","청살모","달다람쥐","저양이","키타","발","아자"]
target_words = group_2point + group_1point

# ====== (중요) 학습 때 썼던 커스텀 레이어 정의 ======
import tensorflow as tf
from tensorflow.keras.layers import Layer
from keras.saving import register_keras_serializable

@register_keras_serializable(package="custom")
class TokenRealMask(Layer):
    def __init__(self, pad_id, **kwargs):
        super().__init__(**kwargs)
        self.pad_id = int(pad_id)
    def call(self, token_ids):
        return tf.not_equal(token_ids, self.pad_id)
    def get_config(self):
        cfg = super().get_config()
        cfg.update({"pad_id": self.pad_id})
        return cfg

@register_keras_serializable(package="custom")
class BuildCrossMask(Layer):
    def call(self, inputs):
        q, tokmask = inputs         # q:(B,Tq,D), tokmask:(B,L) bool
        tokmask = tf.expand_dims(tf.cast(tokmask, tf.bool), axis=1)  # (B,1,L)
        Tq = tf.shape(q)[1]
        return tf.tile(tokmask, [1, Tq, 1])                          # (B,Tq,L)

# ====== 모델 로드 ======
model = load_model(
    MODEL_PATH,
    custom_objects={"TokenRealMask": TokenRealMask, "BuildCrossMask": BuildCrossMask}
)

# ====== Whisper 로드(학습과 동일한 체크포인트 권장) ======
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processor = WhisperProcessor.from_pretrained("openai/whisper-medium")
whisper_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-medium").to(device)
whisper_model.eval()

# pad 토큰 ID (학습 시 PAD_ID = len(tokenizer))
VOCAB_SIZE = len(processor.tokenizer)
PAD_ID = VOCAB_SIZE

# ====== 전처리 ======
def load_wav_to_mel(wav_path, sr=SAMPLE_RATE, n_mels=N_MELS):
    y, _ = librosa.load(wav_path, sr=sr, mono=True)
    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
    mel_db = librosa.power_to_db(mel, ref=np.max)
    mel_norm = (mel_db + 80) / 80.0               # [0,1] 근사
    mel_norm = mel_norm.astype(np.float32)
    return np.expand_dims(mel_norm, axis=-1)      # (128, T, 1)

@torch.no_grad()
def extract_token_ids_from_wav(wav_path):
    speech, _ = librosa.load(wav_path, sr=SAMPLE_RATE, mono=True)
    inputs = processor(speech, sampling_rate=SAMPLE_RATE, return_tensors="pt")
    input_features = inputs.input_features.to(device)
    attn_mask = inputs.attention_mask.to(device) if hasattr(inputs, "attention_mask") and inputs.attention_mask is not None else None

    pred_ids = whisper_model.generate(
        input_features,
        attention_mask=attn_mask,
        language="ko",
        task="transcribe",
        do_sample=False,
        max_new_tokens=MAX_TOKEN_LENGTH,
    )
    token_ids = pred_ids[0].cpu().numpy().astype(np.int32)

    # 학습과 동일하게 PAD_ID로 패딩/자르기
    if token_ids.shape[0] < MAX_TOKEN_LENGTH:
        pad = np.full(MAX_TOKEN_LENGTH - token_ids.shape[0], PAD_ID, dtype=np.int32)
        token_ids = np.concatenate([token_ids, pad], axis=0)
    else:
        token_ids = token_ids[:MAX_TOKEN_LENGTH]
    return token_ids

def prepare_inputs_for_inference(wav_path):
    mel = load_wav_to_mel(wav_path)        # (128, T, 1)
    mel = np.expand_dims(mel, axis=0)      # (1, 128, T, 1)
    tok = extract_token_ids_from_wav(wav_path)
    tok = np.expand_dims(tok, axis=0)      # (1, L)
    return mel, tok

# ====== 점수 출력 ======
def score_audio(wav_path, mdl, threshold=THRESHOLD, label_names=target_words, top_k=10):
    X_mel, X_tok = prepare_inputs_for_inference(wav_path)
    probs = mdl.predict([X_mel, X_tok], verbose=0)[0]   # (num_labels,)
    count = int((probs >= threshold).sum())
    picked = [(w, float(p)) for w, p in zip(label_names, probs) if p >= threshold]
    picked_sorted = sorted(picked, key=lambda x: x[1], reverse=True)

    print(f"총점: {count}")


# ====== 사용 예시 ======

wav_file = r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\1004\CLAP_A\6\p_1_0.wav"
score_audio(wav_file, model, threshold=THRESHOLD)


c:\Users\ous37\my_project\venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\ous37\my_project\venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\ous37\my_project\venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/resource_handle.proto. Please update the gencode to avoid compatibili

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


총점: 9


In [ ]:
# ====== 일괄 처리용 WAV 경로들 ======
wav_paths = [
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\1001\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\1003\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\1004\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\2006\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\2007\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3008\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3009\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3010\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3011\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\2014\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\1015\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3017\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3018\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3019\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3020\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3022\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\1023\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\1024\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\1025\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\1026\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\1027\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3028\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3029\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3030\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\1032\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\1033\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\1034\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\1035\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\1036\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3037\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3038\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3039\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3040\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3041\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\1042\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\1043\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\1045\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\4046\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\4047\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\4052\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\4053\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\4054\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\4055\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\4056\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\4057\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\5058\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\5059\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\5060\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\5061\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\5062\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\5063\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\5064\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\5065\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\5066\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\5067\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3070\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3071\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3074\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3075\CLAP_A\6\p_1_0.wav",
    r"C:\Users\ous37\Downloads\임상data(폴더명 수정)\3076\CLAP_A\6\p_1_0.wav",
]

# ====== 일괄 실행 ======
import os

for idx, wav_file in enumerate(wav_paths, start=1):
    if not os.path.exists(wav_file):
        print(f"[{idx:02d}] 파일 없음: {wav_file}")
        continue
    try:
        print(f"\n[{idx:02d}] {wav_file}")
        score_audio(wav_file, model, threshold=THRESHOLD)
    except Exception as e:
        print(f"[{idx:02d}] 처리 중 오류: {e}")



[01] C:\Users\ous37\Downloads\임상data(폴더명 수정)\1001\CLAP_A\6\p_1_0.wav
총점: 5

[02] C:\Users\ous37\Downloads\임상data(폴더명 수정)\1003\CLAP_A\6\p_1_0.wav
총점: 7

[03] C:\Users\ous37\Downloads\임상data(폴더명 수정)\1004\CLAP_A\6\p_1_0.wav
총점: 9

[04] C:\Users\ous37\Downloads\임상data(폴더명 수정)\2006\CLAP_A\6\p_1_0.wav
총점: 1

[05] C:\Users\ous37\Downloads\임상data(폴더명 수정)\2007\CLAP_A\6\p_1_0.wav
총점: 9

[06] C:\Users\ous37\Downloads\임상data(폴더명 수정)\3008\CLAP_A\6\p_1_0.wav
총점: 12

[07] C:\Users\ous37\Downloads\임상data(폴더명 수정)\3009\CLAP_A\6\p_1_0.wav
총점: 1

[08] C:\Users\ous37\Downloads\임상data(폴더명 수정)\3010\CLAP_A\6\p_1_0.wav
총점: 9

[09] C:\Users\ous37\Downloads\임상data(폴더명 수정)\3011\CLAP_A\6\p_1_0.wav
총점: 13

[10] C:\Users\ous37\Downloads\임상data(폴더명 수정)\2014\CLAP_A\6\p_1_0.wav
총점: 5

[11] C:\Users\ous37\Downloads\임상data(폴더명 수정)\1015\CLAP_A\6\p_1_0.wav
총점: 9

[12] C:\Users\ous37\Downloads\임상data(폴더명 수정)\3017\CLAP_A\6\p_1_0.wav
총점: 2

[13] C:\Users\ous37\Downloads\임상data(폴더명 수정)\3018\CLAP_A\6\p_1_0.wav
총점: 3

[14] C:\U